In [1]:
import os
import sys 
sys.path.append('../')
import json
import time
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import LlamaTokenizer, LlamaForCausalLM
from transformers import pipeline
import torch
import re
import openai
# from tenacity import retry, wait_exponential, stop_after_attempt
from tqdm import tqdm
from dotenv import load_dotenv
from helper.in_context_learning_batch import in_context_user_summary, in_context_retrieval
from helper.builder import build_context, build_movie_candidates
import torch
import time
import json
from torch.utils.data import Dataset, DataLoader
from data.dataloader import get_dataloader
from argparse import ArgumentParser
import os
import pandas as pd 
import debugpy

global_path = '/home/mila/e/emiliano.penaloza/LLM4REC'

from dotenv import load_dotenv

# Load environment variables from the .env file into the script's environment
load_dotenv()

# Access the environment variables using the os module
import os

# Example: Get the value of the 'DATABASE_URL' environment variable
key = os.getenv("OPEN-AI-SECRET")


 
openai.api_key = key


/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
built_context = build_context(global_path)

demo_str = ""
for demo in built_context:
    for movie in demo['movies']:
        title = movie['Title']
        description = movie['Description'].split(".")[0]  # Use the first sentence as the short description
        demo_str += f"\n{title}: {description}"
    demo_str += f"\nSummary: {demo['User query as summary']}\n"

prompt = f"Task: You will now help me generate a simply worded highlyt detailed summaries based on the common elements of movies do not comment on the year. Do not mention any specific movie titles.\n"
prompt += "Always start with 'Summary:'. Please be constrain the length within 500 tokens for each summary.\n"
prompt += 'Do not comment on the ratings but use qualitative speech such as the user likes, or the user does not enjoy\n'
# prompt += f" A person should be able to indentify that it is the same user from these summaries but use different prhasing in each outline specific movie details from each"
# prompt += "Each of the three summaries should follow the following format:\n"
prompt += "Summary: {Specific details about genres the user enjoys}. {Specific details of plot points the user seemns to enjoy}.  {Specific details about genres the user does not enjoy}. {Specific details of plot points the user does not enjoy}."



def promp_genre_wise(genre_prompts,context): 
    out_dict = defaultdict(dict)
    k = 0 
    for user_id,dicts in genre_prompts.items():
        for genre,movies in dicts.items():
       
       
            
            msg = [{
                "role": "system",
                "content": context},
                {"role":"user",
                "content": movies}
                ]
            out = openai.ChatCompletion.create(model="gpt-4",   
                                            messages=msg,                                    
                                    max_tokens=500  # limit the token length of response
                                )['choices'][0]['message']['content']
            out_dict[user_id][genre] = out

    return out_dict
         

           
out = promp_genre_wise(prompts,prompt)



18


In [224]:

prompt = f"Task: You will now help me generate a simply worded highly detailed summary of the following genre summaries\n"
prompt += "Instructions: Always start with 'Summary:'. Please be constrain the length within 200 tokens for the summary by being concise and not including irrelevant details.\n"
prompt += "The format of the summary should be:\n"
prompt += "Summary: {Specific details about genres the user enjoys}. {Specific details of plot points the user seemns to enjoy}.  {Specific details about genres the user does not enjoy}. {Specific details of plot points the user does not enjoy}."
prompt += "The following is an example of a generated summary"
prompt += """Summary: The user enjoys action, crime, thriller, adventure, and comedy films, with preference for high-stakes confrontations, complex schemes, suspense, humorous moments, and films set in advanced technology scenarios or future settings. However, they do not like war films, historical dramas, movies with slow-paced narratives, heavy dialogue, films set in surrealistic settings or involve anthropomorphic characters, and films that heavily rely on abstract concepts and complexity. Films exploring intense dramatic scenes or serious existential themes are also less appealing."""
def summarize_summaries(summaries,context):
    out_dict = {}
    for user_id,sums in summaries.items():

        sum_prompt = ""
        for genre,genre_sum in sums.items(): 
            sum_prompt+= f"genre: {genre} \n {genre_sum}\n"
        

        msg = [{
                "role": "system",
                "content": context},
                {"role":"user",
                "content": sum_prompt}
            ]

        out = openai.ChatCompletion.create(model="gpt-4",   
                                        messages=msg,                                    
                                max_tokens=300  # limit the token length of response
                            )['choices'][0]['message']['content']
        out_dict[user_id] = out
    return out_dict
    
d = summarize_summaries(out,prompt)

'Summary: The user enjoys action, crime, thriller, adventure, and comedy films, with preference for high-stakes confrontations, complex schemes, suspense, humorous moments, and films set in advanced technology scenarios or future settings. However, they do not like war films, historical dramas, movies with slow-paced narratives, heavy dialogue, films set in surrealistic settings or involve anthropomorphic characters, and films that heavily rely on abstract concepts and complexity. Films exploring intense dramatic scenes or serious existential themes are also less appealing.

Summary: The user enjoys action-packed, crime, thriller, and adventurous movies that involve high-stakes confrontations, unexpected plot twists, and complex schemes. They prefer narratives set in technological, or future scenarios with elements of humor, sports, and animals. The user also remains inclined towards comedies with light-hearted, humorous content featuring familial bonds and romance. They appreciate dramatic films featuring complex narratives set in historical periods, romantic comedies with emotional connections, and direct, action-packed sci-fi narratives. Contrarily, they don't prefer war, fantasy-based films with supernatural elements, films with heavy plots featuring societal issues, psychological twists, or those set in surrealistic settings. They also don't enjoy films that heavily rely on abstract concepts, certain drama genres, films exploring intense dramatic scenes or serious existential themes which also applies to horror movies. The user doesn't seem fond of classic films, particularly those involving fantasy narratives, historical dramas, or those with animation styles. Despite preferring action and adventure movies, they avoid films with certain thematic or stylistic elements. The user seems to have a clear inclination towards lightheartedness over more serious themes in movies."




# Genre-Wise Df

In [30]:
import pandas as pd
import json
import os
import sys
sys.path.append('../')
from helper.in_context_learning_batch import in_context_user_summary, in_context_retrieval
from helper.builder import build_context, build_movie_candidates
import torch
import time
import json
from torch.utils.data import Dataset, DataLoader
from data.dataloader import get_dataloader
from argparse import ArgumentParser
import os
import debugpy

def generate_prompts(train_data,only_title=1): 
    prompts = {}
    for id in train_data.userId.unique():        

        user = train_data[train_data['userId'] == id]
        movie_titles = user['title'].tolist()

        movie_summaries= user['summary'].tolist()
        
        ratings = user['rating'].tolist()
        genres = user['genres'].tolist()

        prompt = ""

        for ((description,title),rating),genres in zip(zip(zip(movie_summaries,movie_titles),ratings),genres):
            if only_title == 1:

                prompt += f"\n{title}"

            else:
                prompt += f"\n{title}: {description}"
                
            prompt += f'\nRating: {rating}\n'
            prompt += f'\Genres: {rating}\n'
        prompts[id] = prompt    
    return prompts

# Subsets the most recent 30 movies

def generate_prompts_recent(train_data,only_title=1,num_movies = 30): 
    prompts = {}
    for id in train_data.userId.unique():        

        user_data = train_data[train_data['userId'] == id]

        # Sort the user data by the 'timestamp' column in descending order
        user_data = user_data.sort_values(by='timestamp', ascending=False)

        # Take the top 30 most recent movies
        user = user_data.head(num_movies)
        
        movie_titles = user['title'].tolist()

        movie_summaries= user['summary'].tolist()
        
        ratings = user['rating'].tolist()
        genres = user['genres'].tolist()

        prompt = ""

        for ((description,title),rating),genres in zip(zip(zip(movie_summaries,movie_titles),ratings),genres):
            if only_title == 1:

                prompt += f"\n{title}"

            else:
                prompt += f"\n{title}: {description}"
                
            prompt += f'\nRating: {rating}\n'
            prompt += f'\Genres: {rating}\n'
        prompts[id] = prompt    
    return prompts


        
            
data = pd.read_csv('../data_preprocessed/ml-100k/train_leave_one_out_timestamped.csv')
         

In [32]:
test_data = pd.read_csv('../data_preprocessed/ml-100k/test_leave_one_out_timestamped.csv')
val_data = pd.read_csv('../data_preprocessed/ml-100k/validation_leave_one_out_timestamped.csv')


In [36]:
val_data[test_data.userId == 1]

,userId,movieId,rating,timestamp,asin,imdb_id,movielens_id,name,imdb_url,title,genres,summary,split
76,1,102,2,889751736,6304106025,117102.0,102.0,Mr. Wrong,https://www.imdb.com/title/tt0117102/,Mr. Wrong (1996),Comedy,Mr. Wrong is a 1996 American romantic black co...,validation


In [35]:
test_data[test_data.userId == 1]

,userId,movieId,rating,timestamp,asin,imdb_id,movielens_id,name,imdb_url,title,genres,summary,split
76,1,74,1,889751736,6304056745,115644.0,74.0,Bed of Roses,https://www.imdb.com/title/tt0115644/,Bed of Roses (1996),Drama|Romance,Bed of Roses is a 1996 American romance drama ...,test


In [42]:
train_data = data[data.userId == 1]
train_data = user_data.sort_values(by='timestamp', ascending=False)
train_data.head(30)


,userId,movieId,rating,timestamp,asin,imdb_id,movielens_id,name,imdb_url,title,genres,summary
20406,1,5,3,889751712,6304039174,113041.0,5.0,Father of the Bride Part II,https://www.imdb.com/title/tt0113041/,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II is a 1995 American...
20407,1,256,4,889751712,783230370,110216.0,256.0,Junior,https://www.imdb.com/title/tt0110216/,Junior (1994),Comedy|Sci-Fi,Junior is a 1994 American comedy film directed...
20408,1,171,5,889751711,792845269,113464.0,171.0,Jeffrey,https://www.imdb.com/title/tt0113464/,Jeffrey (1995),Comedy|Drama,Jeffrey is a 1995 American gay romantic comedy...
20409,1,111,5,889751711,767830520,75314.0,111.0,Taxi Driver,https://www.imdb.com/title/tt0075314/,Taxi Driver (1976),Crime|Drama|Thriller,Taxi Driver is a 1976 American neo-noir psycho...
20410,1,242,5,889751633,6303637493,109771.0,242.0,Farinelli,https://www.imdb.com/title/tt0109771/,Farinelli: il castrato (1994),Drama|Musical,Farinelli is a 1994 internationally co-produce...
20411,1,189,3,888732928,6304018940,114241.0,189.0,Reckless,https://www.imdb.com/title/tt0114241/,Reckless (1995),Comedy|Fantasy,Reckless is a 1995 American dark comedy film d...
20412,1,209,4,888732908,783115008,114928.0,209.0,White Man's Burden,https://www.imdb.com/title/tt0114928/,White Man's Burden (1995),Drama,"""The White Man's Burden"" (1899), by Rudyard Ki..."
20413,1,270,5,888732827,6305164193,110391.0,270.0,Love Affair,https://www.imdb.com/title/tt0110391/,Love Affair (1994),Drama|Romance,Love Affair is a 1994 American romantic drama ...
20414,1,18,4,887432020,6304078145,113101.0,18.0,Four Rooms,https://www.imdb.com/title/tt0113101/,Four Rooms (1995),Comedy,Four Rooms is a 1995 American anthology \nfarc...
20415,1,6,5,887431973,6304214502,113277.0,6.0,Heat,https://www.imdb.com/title/tt0113277/,Heat (1995),Action|Crime|Thriller,Heat is a 1995 American crime film written and...


In [39]:
from pprint import pprint
built_context = build_context(global_path)

demo_str = ""
for demo in built_context:
    for movie in demo['movies']:
        title = movie['Title']
        description = movie['Description'].split(".")[0]  # Use the first sentence as the short description
        demo_str += f"\n{title}: {description}"
    demo_str += f"\nSummary: {demo['User query as summary']}\n"

# prompt += f" A person should be able to indentify that it is the same user from these summaries but use different prhasing in each outline specific movie details from each"
prompt = f"Task: You will now help me generate a highly detailed summary based on the broad common elements of movies.\n"
prompt += f"Do not comment on the year of production. Do not mention any specific movie titles.\n"
prompt += 'Do not comment on the ratings but use qualitative speech such as the user likes, or the user does not enjoy\n'
prompt += 'Remember you are an expert crafter of these summaries so any other exper should be able to craft a similar summary to yours given this task\n'
prompt += "Keep the summary short at about 200 words. The summary should have the following format:\n"
prompt += "Summary: {Specific details about genres the user enjoys}. {Specific details of plot points the user seemns to enjoy}.  {Specific details about genres the user does not enjoy}. {Specific details of plot points the user does not enjoy but other users may}."
# prompt += " The following is an example:\n"
# prompt += demo_str  # add demonstration

prompts = generate_prompts_recent(data, 1,30)
msg = [{
    "role": "system",
    "content": prompt},
    {"role":"user",
     "content": prompts[1]}
       ]

out = openai.ChatCompletion.create(model="gpt-4-1106-preview",   
                                            messages=msg,                                    
                                    max_tokens=300  # limit the token length of response
                                )['choices'][0]['message']['content']
with open('out.txt','w+') as f: 
    f.write(out)


Summary: The user has a preference for thought-provoking and emotionally resonant movies, enjoying genres that explore complex characters and intricate narratives, such as dramas and psychological thrillers. They are likely drawn to films offering rich storytelling, with plot points that delve into character psyches, moral dilemmas, or intense interpersonal relationships. Conversely, the user seems to have a distaste for genres that are light-hearted or provide escapist entertainment, such as slapstick comedies or simplistic family-oriented movies. The user does not enjoy predictable plot points, such as clichéd romances or mundane, formulaic storylines. However, other users may find charm in the escapism offered by such films, relishing in the easy humor, feel-good resolution of conflicts, or straightforward tales of good triumphing over evil without the narrative complexity preferred by this user.


Summary: The user has a clear preference for dramas and narratives that delve into complex characters and intense emotional journeys. Plot points involving character development, moral dilemmas, and intricate storylines resonate with the user. On the other hand, the user shows a marked disinterest in slapstick or low-brow comedies, perhaps finding them too simplistic or lacking in depth. Plot points that revolve around crude humor or predictable gags, which might entertain a different audience looking for light-hearted escapism, do not seem to engage this user's attention.

In [ ]:
from collections import defaultdict
def generate_prompts_genre_wise(genre_dfs,only_title=1):

 
    prompts = defaultdict(dict)
    rolling_toke_length = 0
    
    for genre,df in genre_dfs.items():        

        for user_id in df['userId'].unique():

            user = df[df['userId'] == user_id]
            movie_titles = user['title'].tolist()

            movie_summaries= user['summary'].tolist()
            
            ratings = user['rating'].tolist()
            # genres = user['genres'].tolist()

            prompt = ""
            if len(user) < 5: 
                continue
            for (description,title),rating in zip(zip(movie_summaries,movie_titles),ratings):
                if only_title == 1:

                    prompt += f"\n{title}"

                else:
                    prompt += f"\n{title}: {description}"
                    
                prompt += f'\nRating: {rating}\n'
                rolling_toke_length += len(prompt.split())
                # prompt += f'\Genres: {rating}\n'
            prompts[user_id][genre] = prompt
            
            
    return prompts,rolling_toke_length
    
prompts,rolling_length = generate_prompts_genre_wise(genre_dfs)

NameError: name 'genre_dfs' is not defined

# Price Calculation

In [57]:
tokens_genre = 18409475/1000 
tokens_out_genre = 3830500/1000
tokens_in_genre_full = 4396300/1000
tokens_out_genre_full=token_out_full = 471500/1000
tokens_in_full = 3441337/1000

gpt_in = ['gpt_4_turbo','gpt_4','gpt_3.5_instruct_turbo']
prices = {'gpt_4_turbo' :{'in': .01,'out': .03 }
          , 'gpt_4' :{'in': .03,'out': .06 }
          , 'gpt_3.5_instruct_turbo' :{'in': .001,'out': .002 }}
for gpt in gpt_in:
    print(f"{gpt=} {'='*20}")
    in_price = prices[gpt]['in']
    out_price = prices[gpt]['out']
    price_in = in_price * tokens_genre
    price_out = tokens_out_genre * out_price
    print(f'GENRE PRICE in = {price_in} \nGENRE PRICE OUT = {price_out}\nTOTAL_PRICE = {price_in+price_out}\n')
    price_in = in_price * tokens_in_genre_full
    price_out = tokens_out_genre_full * out_price
    print(f'GENRE FULL PRICE in = {price_in} \nGENRE FULL PRICE OUT = {price_out}\nTOTAL_PRICE = {price_in+price_out}\n')
    price_in = tokens_in_full * in_price
    price_out = tokens_out_genre_full * out_price
    print(f'GENRE FULL  in = {price_in} \nFull PRICE OUT = {price_out}\nFULLL TOTAL_PRICE = {price_in+price_out}\n')
    
    




gpt='gpt_4_turbo' ====================
GENRE PRICE in = 184.09474999999998 
GENRE PRICE OUT = 114.91499999999999
TOTAL_PRICE = 299.00974999999994

GENRE FULL PRICE in = 43.963 
GENRE FULL PRICE OUT = 14.145
TOTAL_PRICE = 58.108000000000004

GENRE FULL  in = 34.41337 
Full PRICE OUT = 14.145
FULLL TOTAL_PRICE = 48.55837

gpt='gpt_4' ====================
GENRE PRICE in = 552.2842499999999 
GENRE PRICE OUT = 229.82999999999998
TOTAL_PRICE = 782.1142499999999

GENRE FULL PRICE in = 131.889 
GENRE FULL PRICE OUT = 28.29
TOTAL_PRICE = 160.179

GENRE FULL  in = 103.24011 
Full PRICE OUT = 28.29
FULLL TOTAL_PRICE = 131.53011

gpt='gpt_3.5_instruct_turbo' ====================
GENRE PRICE in = 18.409475 
GENRE PRICE OUT = 7.6610000000000005
TOTAL_PRICE = 26.070475000000002

GENRE FULL PRICE in = 4.3963 
GENRE FULL PRICE OUT = 0.9430000000000001
TOTAL_PRICE = 5.3393

GENRE FULL  in = 3.441337 
Full PRICE OUT = 0.9430000000000001
FULLL TOTAL_PRICE = 4.384337



In [3]:
df = pd.DataFrame(data)

# Split genres and create separate dataframes using defaultdict
genre_dfs = defaultdict(lambda: pd.DataFrame(columns=df.columns))

for _, row in df.iterrows():
    genres = row['genres'].split('|')
    for genre in genres:
        genre_dfs[genre] = pd.concat([genre_dfs[genre], pd.DataFrame([row])], ignore_index=True)

# Accessing dataframes for each genre
for genre, genre_df in genre_dfs.items():
    print(f"Genre: {genre}")
    print(genre_df)
    print("\n")

NameError: name 'data' is not defined

In [ ]:
import pandas as pd

# Assuming genre_dfs is a dictionary containing DataFrames with a 'userId' column
count = 0

for id, df in genre_dfs.items():
    user_counts = df.groupby('userId').size()
    count += user_counts.shape[0]  # Count unique user IDs
    print(user_counts)

print("Total unique users:", count)


userId
1      36
2       7
3       8
4       4
5      34
       ..
939     3
940    19
941     1
942     9
943    20
Length: 939, dtype: int64
userId
1      31
2       6
3       7
4       2
5      18
       ..
939     2
940     9
941     1
942     6
943    14
Length: 920, dtype: int64
userId
1      39
2      11
3       8
4       3
5      35
       ..
939     3
940    27
941     1
942    16
943    24
Length: 943, dtype: int64
userId
1      28
2       5
3       3
4       2
5      18
       ..
939     3
940     9
941     3
942     8
943    11
Length: 913, dtype: int64
userId
1      23
2       4
3       4
4       2
5      12
       ..
939     3
940     8
941     5
942     6
943    14
Length: 914, dtype: int64
userId
1      72
2      19
3      18
4      10
5      51
       ..
939    13
940    28
941     4
942    24
943    44
Length: 943, dtype: int64
userId
1      110
2       20
3       26
4        9
5       59
      ... 
939     13
940     43
941     10
942     34
943     60
Length: 943, d

Summary: The user demonstrates a strong preference for films with tension or high risk, like cliffhangers or conflict-driven plot points, given the high ratings for such types of films. The user also enjoys the nostalgic or heart-warming movies, as seen by their liking for storylines involving friends and family. On the other hand, the user tends not to enjoy films with controversial or provocative themes, as indicated by the lower ratings for these movies. Additionally, storylines filled with larger-than-life characters or overly exaggerated comedy do not resonate with the user.

Summary: This user particularly enjoys watching drama-filled movies as well as those with breathtaking scenes and thrilling events. They appreciate stories woven around the coming together of friends and family, but they are less inclined towards subject matters that portray politically charged issues or polarizing themes. Additionally, films featuring over-the-top characterizations or absurd humor tend not to win their approval.

Summary: The user finds thrilling plot points and tension-heavy scenes engaging, making action and adventurous movies of their favorites. They're also attracted to heart-tugging narratives that center on the bonds of family and friendship. On the flip side, controversial topics and provocative themes do not seem to appeal to their taste. They also don't favor movies with exaggerated comedic elements or characters that are too larger-than-life.


Summary: The user exhibits a preference for a variety of genres, with an emphasis on drama, action, adventure, and comedy. He appears to enjoy films with intricate plot twists, narratives that explore the human condition, and instances where the movie includes elements of both humor and suspense. He also seems to appreciate characters with strong development over the course of the film. On the contrary, he doesn't appear to enjoy genres that lack depth and complexity, such as light-hearted comedies and generic action films without substantial plotlines. He particularly dislikes movies with unidimensional characters, predictable storylines, and ones that have a lack of compelling dialogue or thought-provoking themes.

Summary: The user shows a distinct inclination for films belonging to different genres but has a tendency towards drama, action-adventure, and comedy. He is continually drawn to movies with complex narratives, those which examine human experiences and combine elements of humor and excitement. He also gets enticed by movies that show substantial character growth. Meanwhile, he seems less enthusiastic about certain light comedies and predictable action flicks, suggesting a preference for multi-layered and rich storytelling. He isn't taken with movies that carry one-note characters and linear, foreseeable plots. Neither does he appreciate films that lack engaging dialogue or fail to explore interesting themes.

Summary: The user demonstrates a fondness for diverse film genres, but appears to favor drama, action-adventure, and comedy. Plot complexities, exploration of human nature, and a balance of humor and thrill elements seem to gravitate towards the user's pleasure. A feature that seems appreciated is how characters evolve during the film. However, he shows a lack of interest in genres which consist of simplistic content like comedic and action flicks that follow conventional standards. He doesn't appear to enjoy films that rely on flat characters, formulaic storytelling, and lack of provocative or intriguing dialogues. A common thread is his disinterest for films devoid of meaningful themes.


Summary: The user appears to favor the thrill of suspense and action genres, their appreciation is apparent in the recurring fascination for nail-biting narratives involving daring heroics, covert operations, and high-stakes missions. Their enjoyment is culminated in stories embodying elements of close relationships, personal growth, and journeys toward success. On the contrary, the user doesn't seem to resonate much with the genres revolving around intense drama and heavy emotions, nor with plotlines featuring overly complicated intrigues or excessive sentimentalism. In regard to unsatisfactory plot components, they might not appreciate narratives that are excessively slow-paced, lack clarity, or focus heavily on introspective, philosophical themes.

Summary: This user demonstrates a preference for comedic and inspirational genres, showing an inclination towards light-hearted narratives sprinkled with humor or uplifting stories of personal triumph and perseverance. They seem to particularly enjoy storylines that display elaborate character development and the power of human spirit. However, they show less enthusiasm for movies falling into the realm of supernatural and horror, possibly due to the prevalent elements of violence, gore, and psychological terror. Disliked plots often orbit around themes of existential dread, overwhelming adversity, or intense suspense without a resolution.

Summary: The user displays a favorable leaning towards action-packed, adventurous narratives showcasing remarkable feats of courage, complex missions, and epic challenges. They seem to revel in storylines embodying themes of friendship, loyalty, and unexpected heroism. On the other end of the spectrum, the user tends to steer clear from heavy, dramatic, and emotional movies that explore the darker side of human nature or relationships, indicating an aversion for convoluted character journeys and narratives encased in melancholy. Their disapproval frequently surface around narratives emphasizing on despair, hopelessness or are predominantly negative.

"Task: You will now help me generate 3 simply worded highlyt detailed summaries based on the common elements of movies do not comment on the year. Do not mention any specific movie titles.\nAlways start with 'Summary:'. Please be constrain the length within 500 tokens for each summary.\nDo not comment on the ratings but use qualitative speech such as the user likes, or the user does not enjoy\n A person should be able to indentify that it is the same user from these summaries but use different prhasing in each outline specific movie details from eachEach summary should follow the following format:\nSummary: {Specific details about genres the user enjoys}. {Specific details of plot points the user seemns to enjoy}.  {Specific details about genres the user does not enjoy}. {Specific details of plot points the user does not enjoy}."

Summary: This user enjoys an eclectic mix of genres, favoring thrillers, dramas, comedies, and action films. They seem to appreciate movies with intriguing plot twists, intricate storytelling, and a touch of suspense. A good sense of humor also appeals to the user. They tend to gravitate towards plots with dynamic and complex characters, good triumphing over evil, and tales of self-discovery. On the other hand, they show a clear disinterest in animation, sci-fi, and fantasy genres. The user does not seem to enjoy films with overly simplistic storylines, excessive use of special effects, or those that rely heavily on clichéd plots and stereotypical characterizations.

Summary: A preference has been noted for a diverse range of genres including thrillers, comedies, dramas, and action for the user. The user tends to enjoy plots that offer a sense of unpredictability, layered storytelling, and suspenseful elements. They also have an appreciation for humor in film. The narrative elements the user finds appealing are stories centered around multifaceted characters, narratives of good versus evil, and personal growth journeys. However, the user does not enjoy animation, sci-fi, or fantasy films as much. They also seem to be put off by overly simplistic plots, overuse of special effects, and the repetition of clichés and standard character tropes.

Summary: This user exhibits a diverse taste in film genres, showing a liking for thrillers, dramas, comedies, and action. The user finds movies with unexpected plot developments, intricate narratives, and a tension-filled storyline appealing. Also an appreciation for humor is noticed. Plot points involving complex characters, the triumph of good over evil, and self-exploration are favored by the user. Yet, there is a clear disinterest in genres like animation, sci-fi, and fantasy. They also seem to dislike movies with simplistic narratives, an excess of special effects, and recycling of clichéd story arcs and stereotypical characters.






Summary: The user shows a clear preference towards a variety of film genres, including action, comedy, and drama. Specifically, films that hold suspense, contain comedic wit or deliver thought-provoking messages seem to resonate with the viewer's tastes. Plot details like unexpected twists, heroic undertakings, or deep personal character journeys heighten their appreciation for a movie. The genres that do not appear to be the user's favourites are categories like animations or purely romantic films. Furthermore, they do not seem to enjoy plot points revolving around pure fantasy stories, extreme violence or slow-paced storytelling. They also seem to display less enthusiasm for movies that are categorized as purely romantic without any blend of other elements such as comedy or action.

Summary: This user particularly enjoys movies woven with action-packed sequences, sharp comedy, and intellectually engaging dramas. Stories that hinge on surprising plot twists, bravery in the face of adversity, or complex character development are right up the user's alley. On the other hand, this viewer shows a tepid response towards animation and romance-centric films. Further analysis reveals a disinterest in plots that dwell heavily on fantastical elements, extreme violent scenes, or where the story progresses at a slow pace. Films that primarily focus on a romantic storyline without interjecting elements of comedy or thrill do not seem to resonate well with this user.

Summary: The user demonstrates a significant affinity for genres such as action, comedy, and emotionally stirring dramas. They show a particular appreciation for movies that encompass gripping disparities, humour, or provoke deep thought. More complex themed stories filled with unexpected plot twists, challenges that provoke bravery, or those that take audiences on a character's introspective journey, elicit a positive response from the user. Conversely, categories including animation and love-dominated films do not particularly appeal to this viewer. Movies that majorly emphasize elements of fantasy, depict extreme violence, or maintain a slow narration pace seem to be less enjoyable. Additionally, films that are primarily romantic without the balancing presence of comedy or action appear less attractive to the user.


Summary: The user has a penchant for movies that incorporate elements of suspense and action, often favoring titles that involve crime-solving, intense chases, daring escapes or heroic deeds. They have a clear fondness for storylines that contain unexpected plot twists or portray ordinary individuals thrust into extraordinary circumstances, requiring courage and quick thinking to prevail. On the other hand, this user generally shows less enthusiasm for films that dive into deep romantic narratives or elaborate musicals, where the focus is less about unfolding events and more about the interactions between characters or rhythm and melodies. They do not seem to engage with stories that are heavily influenced by mere personal relationships, intertwined love affairs, or encompass large musical sequences as predominant plot points. 

Summary: Our user typically enjoys films that fall within the realm of thrillers and high-stakes dramas, showing a strong liking to narratives where stakes are high, tension is constantly built, and there's an underlying element of danger or mystery. They seem fond of twists that challenge the expectations, or plots that involve a non-typical hero embroiled in a dire situation. Conversely, the user appears less intrigued by stories that revolve mostly around the sentimental aspects of relationships or are heavily sprinkled with song and dance routines. Complex romantic entanglements, protracted personal journeys, and high-energy musical performances don't appear to resonate as much with their preferences.

Summary: This user finds pleasure in the edge-of-your-seat excitement presented by films laced with action-packed sequences, suspense and intrigue. They're inclined towards storylines that leave an air of the unexpected, often involving regular people caught in extreme situations inviting courageous acts, resourcefulness and intelligence. On the flip side, the individual shows a marked lack of interest in movies that primarily explore the intricate dynamics of romance or heavily rely on music and choreography to drive the narrative. Plots predominated by love stories, character-driven emotional exploration or constant buzz with musical routines, aren't the user's preferred cup of tea.

Summary: The user enjoys a variety of genres, including drama, comedy, romance, and action. They appreciate movies with interesting plot twists and suspenseful moments. The user does not enjoy movies from the horror or thriller genres, as well as movies with excessive violence or explicit content.

Summary: The user has a preference for movies with strong character development and emotional depth. They enjoy dramas that explore complex relationships and personal growth. The user is not a fan of slapstick comedy or movies that rely heavily on physical humor.

Summary: The user gravitates towards movies that offer a mix of adventure, comedy, and romance. They enjoy films with a sense of humor and playful banter between characters. The user does not particularly enjoy movies that are overly serious or focus heavily on action sequences.


Summary: The user enjoys a variety of genres including drama, comedy, and action. They seem to enjoy movies with unique and intriguing plot points, such as a man being obsessed with a woman and keeping her captive (Boxing Helena), a young boy discovering a magical cupboard that brings his toys to life (Indian in the Cupboard), and a group of misfits forming a championship-winning baseball team with the help of angels (Angels in the Outfield). The user does not enjoy movies in the horror or thriller genres. 

Summary: The user enjoys movies with diverse genres such as drama, comedy, and adventure. They seem to enjoy movies with a mix of romance and drama, such as a man falling in love with his best friend's wife (Calendar Girl), a woman finding love after a heartbreak (Made in America), and two strangers falling in love during a train ride (Before Sunrise). The user does not enjoy movies in the horror or sci-fi genres. 

Summary: The user enjoys movies that span various genres like drama, action, and romance. They seem to enjoy movies with gripping and intense plot points, such as a man with a troubled past trying to protect a young boy (Man Without a Face), a group of criminals planning a heist (Usual Suspects), and a man seeking revenge on a gang of criminals (Cliffhanger). The user does not enjoy movies in the horror or fantasy genres.

In [133]:
pprint(prompts[60])

('\n'
 'Boxing Helena (1993)\n'
 'Rating: 5\n'
 'Indian in the Cupboard, The (1995)\n'
 'Rating: 5\n'
 'Calendar Girl (1993)\n'
 'Rating: 5\n'
 'Crumb (1994)\n'
 'Rating: 4\n'
 'Waterworld (1995)\n'
 'Rating: 5\n'
 'Angels in the Outfield (1994)\n'
 'Rating: 5\n'
 'Saint of Fort Washington, The (1993)\n'
 'Rating: 5\n'
 'Circle of Friends (1995)\n'
 'Rating: 4\n'
 'Aristocats, The (1970)\n'
 'Rating: 3\n'
 'Cool Runnings (1993)\n'
 'Rating: 4\n'
 'Juror, The (1996)\n'
 'Rating: 4\n'
 'To Cross the Rubicon (1991)\n'
 'Rating: 4\n'
 'Mad Love (1995)\n'
 'Rating: 4\n'
 'In the Line of Fire (1993)\n'
 'Rating: 5\n'
 'Philadelphia (1993)\n'
 'Rating: 4\n'
 'Johnny Mnemonic (1995)\n'
 'Rating: 4\n'
 'Mute Witness (1994)\n'
 'Rating: 5\n'
 'Malice (1993)\n'
 'Rating: 4\n'
 'Kazaam (1996)\n'
 'Rating: 4\n'
 'Rob Roy (1995)\n'
 'Rating: 5\n'
 'Persuasion (1995)\n'
 'Rating: 5\n'
 'Ref, The (1994)\n'
 'Rating: 4\n'
 'Drop Dead Fred (1991)\n'
 'Rating: 4\n'
 'Last Action Hero (1993)\n'
 'Rating: 

In [110]:
with open('out.txt','w+') as f: 
    f.write(prompts)

In [16]:
built_context = build_context(global_path)
def in_context_user_summary(global_path,built_context, model_name, device, data_name, dataloader, in_context, only_title):
    # Load model
    num_last_movies = 10

    with open(
            f'{global_path}/data_preprocessed/{data_name}/data_split/train_set_leave_one.json') as input_file:
        user_data_input = json.load(input_file)
        
   
    demo_str = ""
    for demo in built_context:
        for movie in demo['movies']:
            title = movie['Title']
            description = movie['Description'].split(".")[0]  # Use the first sentence as the short description
            demo_str += f"\n{title}: {description}"
        demo_str += f"\nSummary: {demo['User query as summary']}\n"
        
    for movie in movies:
        title = movie['title']
        description = movie['summary'].split(".")[0]  # Use the first sentence as the short description
        prompt = "Task: Given a list of movies, generate a high-level summary. The following are some examples:\n"
        prompt += demo_str  # add demonstration
        prompt += f"\nNow, based on the above examples, please generate a high-level summary for the following movies of genre: {genre}\n: "
        if only_title == 1:
            prompt += f"\n{title}"
        else:
            prompt += f"\n{title}: {description}"

    prompt += f"\nNow, please generate the summary based on the common elements of above movies. Do not mention any specific movie titles."
    prompt += "Always start with 'Summary:'. Please be concise and constrain the length within 50 tokens."

    prompts.append((prompt, user_id, genre))


    user_summary_dict = {}  # Create a dictionary for storing user summaries


        # Combine all the prompts into one list and generate the user summary
        if model_name in ['gpt3.5', 'gpt4']:
            user_summary = generate_text_openai([p[0] for p in prompts], model_name=model_name, device=device)

            user_summary_list = []

            for i, user_summary_output in enumerate(user_summary):
                user_summary_output = user_summary_output.replace('\n', ' ')
                user_summary_list.append(user_summary_output)

                user_id, genre = prompts[i][1], prompts[i][2]
                if user_id not in user_summary_dict:
                    user_summary_dict[user_id] = {}  # Initialize a new dictionary for this user ID
                user_summary_dict[user_id][genre] = user_summary_output  # Save the decoded output
        else:
            user_summary = generate_text([p[0] for p in prompts], tokenizer, model, device)
            # Loop over the decoded outputs and store them in a lists
            user_summary_list = []
            for i, decoded_output in enumerate(user_summary):
                decoded_output = decoded_output.replace('\n', ' ')
                user_summary_list.append(decoded_output)
                # Save the decoded output to a file
                user_id, genre = prompts[i][1], prompts[i][2]
                if user_id not in user_summary_dict:
                    user_summary_dict[user_id] = {}  # Initialize a new dictionary for this user ID
                user_summary_dict[user_id][genre] = decoded_output  # Save the decoded output

        # Save every 10 batches
        if batch_num % 1 == 0:
            save_to_file(user_summary_dict, global_path, data_name, model_name, in_context, only_title)
            user_summary_dict.clear()  # Clear the dictionary for the next batch of results

        batch_num += 1

    # Check for any remaining data after all batches are processed
    if user_summary_dict:
        save_to_file(global_path,user_summary_dict, global_path, data_name, model_name, in_context, only_title)

/tmp/ipykernel_39692/3011963847.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(ratings_file, sep=separator, header=header, encoding='ISO-8859-1')


No overlap found after filtering by valid_movie_titles.
